## A demo using Hydrogen Hamiltonian with GPT-QE.

In [7]:
import torch
from gqe.mingpt.utils import set_seed


In [8]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.util import get_device
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
bond_length = 3.0
geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {bond_length}\n" + f"H 0.0 0.0 {2 * bond_length}\n"
molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
nqubit = 12

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=4)
pool = DefaultOperatorPool(paulis)
cost = EnergyCost(hamiltonian, initializer, pool,
                  [1 / 320, -1 / 320, 1 / 160, -1 / 160, 1 / 80, -1 / 80, 1 / 40, -1 / 40, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2])


converged SCF energy = -15.0242100060364
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
paulis [+IIIYZXIIIIII, +IIIYZZZXIIII, +IIIYZZZZZXII, +IIIYZZZZZZZX, +IIXXIIIIIIYX, +IIXXIIIIYXII, +IIXXIIYXIIII, +IIXXYXIIIIII,

## FCI energy by diagonalization

In [9]:
from qwrapper.hamiltonian import compute_ground_state

print(compute_ground_state(hamiltonian))

-15.336459383306721


In [10]:
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(12, [], "qulacs")))

hf state: -15.024210006036476


## Setup for GPT

In [11]:
# create a GPT instance
from gqe.mingpt.model import GPT
from gqe.mingpt.trainer import Trainer


def run(seed):
    set_seed(seed)
    model_config = GPT.get_default_config()
    model_config.model_type = 'gpt2'
    model_config.vocab_size = cost.vocab_size()
    model_config.n_gates = 40  # The number of gates for each circuit
    model_config.block_size = model_config.n_gates
    model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
    model_config.embd_pdrop = 0.1
    model_config.resid_pdrop = 0.1
    model_config.attn_pdrop = 0.1
    model_config.std = 0.02
    model_config.energy_offset = 14
    model = GPT(model_config, cost)

    train_config = Trainer.get_default_config()
    train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
    train_config.max_iters = 300
    train_config.num_workers = 10
    train_config.n_samples = 50
    trainer = Trainer(train_config, model)
    file_monitor = FileMonitor()
    callback_generator = DefaultCallback(model, monitors=[PrintMonitor(), file_monitor], del_temperature=0.01)
    trainer.set_callback('on_batch_end', callback_generator.generate())
    trainer.run()
    # torch.save(model.state_dict(), '../saved_models/gptqe_test_2')
    file_monitor.save(f'../output/trajectory_beh2_{model_config.n_gates}_{seed}.json')

In [12]:
for seed in [31, 37, 43, 47, 53]:
    run(seed)

number of parameters: 85.91M
running on device mps
iter_dt 0.00s; iter 0: train loss 0.39236 temperature: 5
mean_logits tensor([-14.9993, -15.1921, -15.2264, -15.2095, -14.8549, -14.9058, -15.0863,
        -15.0300, -15.0804, -14.9182, -15.1555, -14.9267, -15.0218, -15.0428,
        -15.0164, -14.9592, -14.8410, -15.3652, -15.0018, -15.0925, -15.1750,
        -15.0018, -14.9717, -15.1248, -15.1591, -14.9477, -14.8711, -14.8943,
        -15.0079, -15.0835, -14.8407, -15.0131, -15.3175, -14.8990, -14.9557,
        -15.0355, -15.0282, -14.9517, -15.0035, -15.1420, -14.9938, -15.2316,
        -15.2516, -14.9208, -15.1069, -14.8844, -14.9746, -15.0219, -14.9587,
        -14.8292], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-14.7553, -14.7910, -14.8738, -14.8952, -15.0165, -14.9429, -14.9094,
        -14.9261, -14.9089, -14.7971, -14.9437, -14.9937, -14.8592, -14.8009,
        -14.8948, -14.8116, -14.9219, -14.7436, -14.8981, -14.9423, -14.8033,
        -14.9777, -14.9109, -14